# Convolution

We are g

In [1]:
%load_ext autoreload
%autoreload 2
import sys

! pip install  "sep_plot @ git+https://github.com/SEP-software/sep-plot.git@2c0d7859a30cc97d8cd18139655c54c97cdb2615" 
import generic_solver

  Cloning https://github.com/SEP-software/sep-plot.git (to revision 2c0d7859a30cc97d8cd18139655c54c97cdb2615) to /private/var/folders/69/z1qrsb455x90d4pvp67r4ccr00lgnl/T/pip-install-2h96j_wv/sep-plot_2791e13921874b6ab07c007c006c6919
  Running command git clone -q https://github.com/SEP-software/sep-plot.git /private/var/folders/69/z1qrsb455x90d4pvp67r4ccr00lgnl/T/pip-install-2h96j_wv/sep-plot_2791e13921874b6ab07c007c006c6919
  Running command git rev-parse -q --verify 'sha^2c0d7859a30cc97d8cd18139655c54c97cdb2615'
  Running command git fetch -q https://github.com/SEP-software/sep-plot.git 2c0d7859a30cc97d8cd18139655c54c97cdb2615
  Resolved https://github.com/SEP-software/sep-plot.git to commit 2c0d7859a30cc97d8cd18139655c54c97cdb2615
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Cloning https://github.com/SEP-software/sep-python.git (to revision 33d0fa36811f8d80c4abc2b1e7a35c40d006e3e9) to /private/var/fo

WARNING! DATAPATH not found. The folder /tmp will be used to write binary files


In [4]:
from generic_solver._pyOperator import Operator
from sep_python import FloatVector
import numba

class TconvM(Operator):

    def __init__(self, mod, dat, flt):
        """
        Initialize a transient convolition

            mod, dat - sepVector
            flt - Filter coeficient sepVector
        """
        super().__init__(mod, dat)
        if not isinstance(mod, FloatVector) or not isinstance(dat,FloatVector) or not isinstance(flt,FloatVector):
            raise Exception("Expecting model, data, flt to be sepVectors")
        self._flt=flt.clone()
        self._nm=mod.getHyper().axes[0].n
        self._nd=dat.getHyper().axes[0].n
        self._nb=flt.getHyper().axes[0].n
        if nd != nm+nb-1:
            raise Exception("Expecting nm+nb-1=nd")

    def forward(self, add, mod, dat):
        """
        Forward operation
        """
        self.checkDomainRange(mod, dat)
        if not add:
            dat.zero()
        for im in range(self._nm):
            for ib in range(self._nb):
                dat[ib+im-1]+=mod[im]*self._flt[ib]


    def adjoint(self, add, mod, dat):
        """
        Adjoint operation.
        """
        self.checkDomainRange(mod, dat)
        if not add:
            mod.zero()

        for im in range(self._nm):
            for ib in range(self._nb):
                mod[im]+=dat[ib+im-1]*self._flt[ib]


In [5]:
class TconvF(Operator):

    def __init__(self, mod, dat, flt):
        """
        Initialize a transient convolition

            mod, dat - sepVector
            flt - Filter coeficient sepVector
        """
        super().__init__(flt, dat)
        if not isinstance(mod, FloatVector) or not isinstance(dat,FloatVector) or not isinstance(flt,FloatVector):
            raise Exception("Expecting model, data, flt to be sepVectors")
        self._mod=mod.clone()
        self._nm=mod.getHyper().axes[0].n
        self._nd=dat.getHyper().axes[0].n
        self._nb=flt.getHyper().axes[0].n
        if nd != nm+nb-1:
            raise Exception("Expecting nm+nb-1=nd")

    def forward(self, add, flt, dat):
        """
        Forward operation
        """
        self.checkDomainRange(flt, dat)
        if not add:
            dat.zero()
        for im in range(self._nm):
            for ib in range(self._nb):
                dat[ib+im-1]+=self._mod[im]*flt[ib]


    def adjoint(self, add, mod, dat):
        """
        Adjoint operation.
        """
        self.checkDomainRange(mod, dat)
        if not add:
            mod.zero()

        for im in range(self._nm):
            for ib in range(self._nb):
                flt[ib]+=dat[ib+im-1]*self._mod[im]


In [ ]:
from generic_solver._pyOperator import Operator
from sep_python import FloatVector
import numba

class IconvM(Operator):

    def __init__(self, mod, dat, flt):
        """
        Initialize a transient convolition

            mod, dat - sepVector
            flt - Filter coeficient sepVector
        """
        super().__init__(mod, dat)
        if not isinstance(mod, FloatVector) or not isinstance(dat,FloatVector) or not isinstance(flt,FloatVector):
            raise Exception("Expecting model, data, flt to be sepVectors")
        self._flt=flt.clone()
        self._nm=mod.getHyper().axes[0].n
        self._nd=dat.getHyper().axes[0].n
        self._nb=flt.getHyper().axes[0].n
        if nd != nm:
            raise Exception("Expecting nm=nd")

    def forward(self, add, mod, dat):
        """
        Forward operation
        """
        self.checkDomainRange(mod, dat)
        if not add:
            dat.zero()
        for id in self._nb-1,self._nd
            for ib in self._nb:
                dat[id]+=mod[id-ib]*self._flt[ib]

    def adjoint(self, add, mod, dat):
        """
        Adjoint operation.
        """
        self.checkDomainRange(mod, dat)
        if not add:
            mod.zero()

        for id in self._nd:
            for ib in self._nb:
                mod[id-ib]+=dat[id]*self._flt[ib]
